In [4]:
!pip install deap

118988.71s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [ ]:
!pip install matplotlib seaborn

: 

: 

In [6]:
import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# gr*.json contains the distance map in list of list style in JSON format
# Optimal solutions are : gr17 = 2085, gr24 = 1272, gr120 = 6942
with open("tsp/gr120.json", "r") as tsp_data:
    tsp = json.load(tsp_data)

distance_map = tsp["DistanceMatrix"]
IND_SIZE = tsp["TourSize"]

# creator is a metafactory, which builds the components we compose using .create
# FitnessMin is a "created" hook into deap's fitness prop that can work to minimize (-1) or maxmize (1) the score.
# This works as a synthetic prop we can hook into our chromosomes. 
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Indiv is a "created" object that is an array of signed ints, with a fitness score.
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
# "creates" an indices function that populates an array IND_SIZE (17,24,120) adjusted for ob1** times, consisting of every number in IND_SIZE (never repeating, thanks to random.sample)
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

# Structure initializers
# metafunction called indiv, which is a single individual consisting of an array of signed ints, containing a route of indices
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
# a list of indivs pop:n long.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# Major logic container**
def evalTSP(individual):
    # start with [[last element, peek in stack], [first elem]]
    distance = distance_map[individual[-1]][individual[0]]
    # take consecutive cities starting with the first element between 0(first):-1(last element), and the element after that, the zip function handles the logic of iterations, but is bounds exclusive (meaning it does not sum 0 and -1 with their appropriate pairs), hence the initialisation
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2] # sum distance between cities in order of route
    return distance, # return singleton (tuple that contains only one set of data). This is because deap has multi-object optmisation for multiple fitness values, necessitating a tuple to compute. Due to the nature of this program, only one fitness eval is required, but passing a standard int will flag an issue.


# partiallymatched is standard for tsp, it works to shuffle indices, without creating invalid offsprings
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)

def main():
    random_seed = 0.81090281647037534324
    print("using seed:")
    print(random_seed)
    
    # Global SS = random_seed for debugging. 
    random.seed(random_seed)

    pop = toolbox.population(n=65)

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, 0.981, 0.42, 33204, stats=stats,
                        halloffame=hof, verbose=True)

    return pop, stats, hof

if __name__ == "__main__":
    main()

/Users/nihal_dharmaraj/Documents/Year 3/Python/genetics/TSP_/.venv/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/nihal_dharmaraj/Documents/Year 3/Python/genetics/TSP_/.venv/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


using seed:
0.8109028164703753
gen	nevals	avg    	std   	min  	max  
0  	65    	52348.8	2564.8	45275	57121
1  	65    	51509.2	2732.12	42512	56258
2  	65    	51079.3	2505.42	42755	56769
3  	65    	51108.2	2190.21	44833	55738
4  	65    	50328.1	2015.43	46327	55067
5  	64    	50790.8	2957.39	44018	58248
6  	64    	50763.4	2065.78	45714	57076
7  	65    	50886.8	2591.07	45543	56672
8  	64    	50726.7	2488.39	44792	55956
9  	64    	50342.9	2763.12	42996	57226
10 	62    	50231.8	2296.06	46051	57005
11 	64    	49966.6	2369.66	45446	56718
12 	65    	49950.2	2335.05	45446	56095
13 	63    	50006.6	2457.75	43604	55949
14 	64    	50182.5	2378.79	45155	56358
15 	65    	50990.3	2511.74	45145	58343
16 	63    	50157.8	2559.26	43101	55694
17 	60    	49328.2	2546.17	42512	55357
18 	65    	49008.5	2429.49	43725	54887
19 	62    	49816.3	3005.56	43402	58332
20 	64    	49895.2	2659.4 	43355	55958
21 	64    	50061.1	2442.39	44127	54787
22 	64    	49667.6	2536.93	43587	56451
23 	64    	49419  	2626.12	43175	56